In [1]:
import pandas as pd

# Load dataset
df = pd.read_csv("../data/Aluminum_Good_Bad_30mins.csv")

# Bad Conditioned Nozzle
b_start_time = "2025-01-07 07:14:00+0000"
b_end_time = "2025-01-07 07:43:59+0000"
b_df = df[(df["TimeStamp"] >= b_start_time) & (df["TimeStamp"] <= b_end_time)].reset_index(drop=True)

# Bad Conditioned Nozzle
g_start_time = "2025-01-07 08:16:00+0000"
g_end_time = "2025-01-07 08:45:59+0000"
g_df = df[(df["TimeStamp"] >= g_start_time) & (df["TimeStamp"] <= g_end_time)].reset_index(drop=True)

b_df['Nozzle_Condition'] = 0
g_df['Nozzle_Condition'] = 1

print(f"Number of bad data points: {len(b_df)}")
print(f"Number of good data points: {len(g_df)}")


Number of bad data points: 1770
Number of good data points: 1784


In [2]:
import torch.nn as nn

class LSTMAnomalyDetector(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LSTMAnomalyDetector, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.dense = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(p=0.3)  # 30% dropout rate

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        lstm_out = self.dropout(lstm_out)  # Apply dropout to LSTM output
        output = self.dense(lstm_out[:, -1, :])
        return output


In [3]:
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Define the relevant features
pros_param = ["Chamber_Pressure", "Main_Gas_Flow", "PF2_Pressure", "PF2_Gas_Flow"]

# Assuming g_df contains the good dataset and pros_param lists the relevant features
X = g_df[pros_param].values  # Extract features
y = np.zeros(len(X))  # Assuming all data is normal (good)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Original X_train shape: {X_train.shape}")
print(f"Original X_test shape: {X_test.shape}")

# Normalize data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

# Function to create time-series data
def create_time_series(data, timesteps):
    samples = []
    for i in range(len(data) - timesteps + 1):
        samples.append(data[i:i + timesteps])
    return np.array(samples)

# Define the number of timesteps
timesteps = 60

# Create time-series data for train and test sets
X_train_series = create_time_series(X_train, timesteps)
X_test_series = create_time_series(X_test, timesteps)

# Adjust labels to match the windows (if needed)
y_train_series = y_train[timesteps - 1:]
y_test_series = y_test[timesteps - 1:]

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_series, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_series, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_series, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_series, dtype=torch.float32)

# Print PyTorch tensor shapes
print(f"X_train_tensor shape: {X_train_tensor.shape}")
print(f"X_test_tensor shape: {X_test_tensor.shape}")
print(f"y_train_tensor shape: {y_train_tensor.shape}")
print(f"y_test_tensor shape: {y_test_tensor.shape}")


Original X_train shape: (1427, 4)
Original X_test shape: (357, 4)
X_train_tensor shape: torch.Size([1368, 60, 4])
X_test_tensor shape: torch.Size([298, 60, 4])
y_train_tensor shape: torch.Size([1368])
y_test_tensor shape: torch.Size([298])


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [9]:
# Parameters
input_dim = X_train_series.shape[2]  # Number of features
hidden_dim = 64
output_dim = input_dim
learning_rate = 0.0001
num_epochs = 50
batch_size = 32

# Model, loss function, and optimizer
model = LSTMAnomalyDetector(input_dim, hidden_dim, output_dim)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

# Create DataLoaders
train_loader = torch.utils.data.DataLoader(X_train_tensor, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(X_test_tensor, batch_size=batch_size, shuffle=False)

best_loss = float('inf')
patience = 5
trigger_count = 0

print(f"LSTM Model running on {device}")

for epoch in range(num_epochs):
    model.to(device)
    model.train()
    train_loss = 0.0
    for batch in train_loader:
        batch = batch.to(device)
        # Training step
        optimizer.zero_grad()
        output = model(batch)
        loss = criterion(output, batch[:, -1, :])
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    # Validation step
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in test_loader:
            batch = batch.to(device)
            output = model(batch)
            loss = criterion(output, batch[:, -1, :])
            val_loss += loss.item()
    
    val_loss /= len(test_loader)
    print(f"Epoch {epoch + 1}, Train Loss: {train_loss / len(train_loader):.4f}, Val Loss: {val_loss:.4f}")
    
    # Check early stopping condition
    if val_loss < best_loss:
        best_loss = val_loss
        trigger_count = 0  # Reset patience counter
        best_model = model.state_dict()  # Save the best model
    else:
        trigger_count += 1
        if trigger_count >= patience:
            print("Early stopping triggered!")
            model.load_state_dict(best_model)  # Restore the best model
            break

LSTM Model running on cuda
Epoch 1, Train Loss: 0.2853, Val Loss: 0.2556
Epoch 2, Train Loss: 0.2305, Val Loss: 0.1933
Epoch 3, Train Loss: 0.1421, Val Loss: 0.0593
Epoch 4, Train Loss: 0.0477, Val Loss: 0.0331
Epoch 5, Train Loss: 0.0409, Val Loss: 0.0327
Epoch 6, Train Loss: 0.0402, Val Loss: 0.0327
Epoch 7, Train Loss: 0.0395, Val Loss: 0.0324
Epoch 8, Train Loss: 0.0392, Val Loss: 0.0322
Epoch 9, Train Loss: 0.0384, Val Loss: 0.0319
Epoch 10, Train Loss: 0.0380, Val Loss: 0.0317
Epoch 11, Train Loss: 0.0370, Val Loss: 0.0315
Epoch 12, Train Loss: 0.0364, Val Loss: 0.0314
Epoch 13, Train Loss: 0.0359, Val Loss: 0.0313
Epoch 14, Train Loss: 0.0365, Val Loss: 0.0311
Epoch 15, Train Loss: 0.0368, Val Loss: 0.0312
Epoch 16, Train Loss: 0.0355, Val Loss: 0.0308
Epoch 17, Train Loss: 0.0346, Val Loss: 0.0307
Epoch 18, Train Loss: 0.0348, Val Loss: 0.0306
Epoch 19, Train Loss: 0.0346, Val Loss: 0.0302
Epoch 20, Train Loss: 0.0341, Val Loss: 0.0300
Epoch 21, Train Loss: 0.0338, Val Loss: 0.

In [5]:
# model_save_path = "../trained_model/lstm_aluminium_anomaly.pth"
# torch.save(best_model, model_save_path)
# print(f"Model checkpoint saved to {model_save_path}")

In [ ]:
# Load the trained model
model_save_path = "../trained_model/lstm_aluminium_anomaly.pth"
model.load_state_dict(torch.load(model_save_path))

In [ ]:
# Evaluation
model.eval()
with torch.no_grad():
    reconstructed = model(X_test_tensor)
    reconstruction_error = torch.mean(torch.abs(X_test_tensor[:, -1, :] - reconstructed), axis=1)

# Compute thresholds for slight and heavy abnormalities
slight_abnormal_threshold = np.percentile(reconstruction_error.numpy(), 90)  # 90th percentile
heavy_abnormal_threshold = np.percentile(reconstruction_error.numpy(), 99)  # 99th percentile
print(f"Slight Abnormal Threshold: {slight_abnormal_threshold:.4f}")
print(f"Heavy Abnormal Threshold: {heavy_abnormal_threshold:.4f}")

# Identify anomalies
slight_abnormal_indices = reconstruction_error.numpy() > slight_abnormal_threshold
heavy_abnormal_indices = reconstruction_error.numpy() > heavy_abnormal_threshold



In [8]:
# import matplotlib.pyplot as plt

# # Extract relevant features
# X_unseen = b_df[pros_param].values  # Extract features
# X_unseen = scaler.transform(X_unseen)  # Normalize using the same scaler fitted on the training data
# X_unseen_series = create_time_series(X_unseen, timesteps)  # Convert to time-series format
# X_unseen_tensor = torch.tensor(X_unseen_series, dtype=torch.float32)

# original_indices = b_df.index[timesteps - 1:]  # Map back to original indices

# # Inference with the model
# model.eval()
# with torch.no_grad():
#     reconstructed_new = model(X_unseen_tensor)  # Reconstruct the data
#     reconstruction_error_new = torch.mean(torch.abs(X_unseen_tensor[:, -1, :] - reconstructed_new), axis=1)

# # Detect anomalies
# slight_abnormal_indices = reconstruction_error_new.numpy() > slight_abnormal_threshold
# heavy_abnormal_indices = reconstruction_error_new.numpy() > heavy_abnormal_threshold

# # Map indices back to the original DataFrame
# slight_abnormal_original = original_indices[slight_abnormal_indices]
# heavy_abnormal_original = original_indices[heavy_abnormal_indices]

# # Initialize all labels as 'Normal'
# b_df['Label'] = 'Normal'

# # Update labels for slight and heavy abnormalities
# b_df.loc[slight_abnormal_original, 'Label'] = 'Slight Abnormal'
# b_df.loc[heavy_abnormal_original, 'Label'] = 'Heavy Abnormal'

# # Plot each feature with abnormalities highlighted
# plt.figure(figsize=(14, 4 * len(pros_param)))  # Dynamically adjust height based on the number of features

# for i, feature in enumerate(pros_param):
#     ax = plt.subplot(len(pros_param), 1, i + 1)
    
#     # Plot the full feature trend as a grey line
#     ax.plot(b_df['TimeStamp'], b_df[feature], label=f'{feature} (Full Trend)', linewidth=0.7, color='grey', zorder=0)
    
#     # Normal data points
#     normal_data = b_df[b_df['Label'] == 'Normal']
#     ax.scatter(normal_data['TimeStamp'], normal_data[feature],
#                color='green', label='Normal', marker='.', s=20)
    
#     # Slight abnormal points
#     slight_abnormal_data = b_df[b_df['Label'] == 'Slight Abnormal']
#     ax.scatter(slight_abnormal_data['TimeStamp'], slight_abnormal_data[feature],
#                color='orange', label='Slight Abnormal', marker='o', s=40)
    
#     # Heavy abnormal points
#     heavy_abnormal_data = b_df[b_df['Label'] == 'Heavy Abnormal']
#     ax.scatter(heavy_abnormal_data['TimeStamp'], heavy_abnormal_data[feature],
#                color='red', label='Heavy Abnormal', marker='x', s=60)
    
#     # Set titles and labels
#     ax.set_title(f"{feature} with Abnormalities", fontsize=14)
#     ax.set_xlabel("Time", fontsize=12)
#     ax.set_ylabel("Value", fontsize=12)
#     ax.grid(alpha=0.3)
    

# # Adjust layout for better visualization
# plt.tight_layout()
# plt.show()


In [9]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates


def analyze_spray_data(df, start_time, end_time, spray_start_time, spray_end_time, model, scaler, pros_param, timesteps, slight_abnormal_threshold, heavy_abnormal_threshold, nozzle_condition):
    """
    Analyze spray data, detect anomalies, and plot results.

    Parameters:
    - df (DataFrame): Input DataFrame containing the time series data.
    - start_time (str): Start time for filtering the dataset.
    - end_time (str): End time for filtering the dataset.
    - spray_start_time (str): Start time of the spray process.
    - spray_end_time (str): End time of the spray process.
    - model: Pre-trained PyTorch model for anomaly detection.
    - scaler: Scaler used to normalize the features.
    - pros_param (list): List of feature column names to analyze.
    - timesteps (int): Number of timesteps for the time-series window.
    - slight_abnormal_threshold (float): Threshold for slight abnormalities.
    - heavy_abnormal_threshold (float): Threshold for heavy abnormalities.

    Returns:
    None
    """

    df['TimeStamp'] = pd.to_datetime(df['TimeStamp'], format="%Y%m%dT%H:%M:%S.%f")

    # Convert string times to datetime
    start_time = pd.to_datetime(start_time, format="%Y%m%dT%H:%M:%S.%f")
    end_time = pd.to_datetime(end_time, format="%Y%m%dT%H:%M:%S.%f")
    spray_start_time = pd.to_datetime(spray_start_time, format="%Y%m%dT%H:%M:%S.%f")
    spray_end_time = pd.to_datetime(spray_end_time, format="%Y%m%dT%H:%M:%S.%f")

    # Filter the DataFrame to the defined time range
    df = df[(df['TimeStamp'] >= start_time) & (df['TimeStamp'] <= end_time)].reset_index(drop=True)
    spray_df = df[(df['TimeStamp'] >= spray_start_time) & (df['TimeStamp'] <= spray_end_time)].reset_index(drop=True)

    # Extract relevant features
    X_unseen = df[pros_param].values  # Extract features
    X_unseen_spray = spray_df[pros_param].values
    scaler = MinMaxScaler()
    X_unseen_scalar = scaler.fit(X_unseen_spray)
    X_unseen = X_unseen_scalar.transform(X_unseen)  # Normalize using the fitted scaler
    X_unseen_series = create_time_series(X_unseen, timesteps)  # Convert to time-series format
    X_unseen_tensor = torch.tensor(X_unseen_series, dtype=torch.float32)

    # Maintain mapping between time-series windows and original indices
    original_indices = df.index[timesteps - 1:]

    # Inference
    model.eval()
    with torch.no_grad():
        reconstructed_new = model(X_unseen_tensor)  # Reconstruct the new data
        reconstruction_error_new = torch.mean(torch.abs(X_unseen_tensor[:, -1, :] - reconstructed_new), axis=1)

    # Detect anomalies
    slight_abnormal_indices = reconstruction_error_new.numpy() > slight_abnormal_threshold
    heavy_abnormal_indices = reconstruction_error_new.numpy() > heavy_abnormal_threshold

    # Map indices back to the original DataFrame
    slight_abnormal_original = original_indices[slight_abnormal_indices]
    heavy_abnormal_original = original_indices[heavy_abnormal_indices]

    # Initialize the 'Label' column as 'Not Evaluated'
    df['Label'] = 'Not Evaluated'

    # Set 'Label' to 'Normal' starting from the timesteps index onwards
    df.loc[timesteps:, 'Label'] = 'Normal'

    # Update labels for slight and heavy abnormalities
    df.loc[slight_abnormal_original, 'Label'] = 'Slight Abnormal'
    df.loc[heavy_abnormal_original, 'Label'] = 'Heavy Abnormal'

    # Count abnormalities within spray start and end times
    spray_data = df[(df['TimeStamp'] >= spray_start_time) & (df['TimeStamp'] <= spray_end_time)]
    normal_count = spray_data[spray_data['Label'] == 'Normal'].shape[0]
    slight_abnormal_count = spray_data[spray_data['Label'] == 'Slight Abnormal'].shape[0]
    heavy_abnormal_count = spray_data[spray_data['Label'] == 'Heavy Abnormal'].shape[0]

    total_data = normal_count + slight_abnormal_count + heavy_abnormal_count

    print(f"Normal Count within Spray Time: {normal_count}")
    print(f"Slight Abnormal Count within Spray Time: {slight_abnormal_count}")
    print(f"Heavy Abnormal Count within Spray Time: {heavy_abnormal_count}")

    # Plot the features with abnormalities highlighted
    plt.figure(figsize=(14, 20))

    for i, feature in enumerate(pros_param):
        plt.subplot(len(pros_param), 1, i + 1)
        plt.plot(df['TimeStamp'], df[feature], label=feature, linewidth=0.7, color='grey', zorder=0)

        # Normal data
        normal_data = df[df['Label'] == 'Normal']
        plt.scatter(normal_data['TimeStamp'], normal_data[feature],
                    color='green', label=f'Normal (N={normal_count}, {(normal_count/total_data)*100:.1f}%))', marker='o', s=20)

        # Slight Abnormal
        slight_abnormal_data = df[df['Label'] == 'Slight Abnormal']
        plt.scatter(slight_abnormal_data['TimeStamp'], slight_abnormal_data[feature],
                    color='orange', label=f'Slight Abnormal (N={slight_abnormal_count}, {(slight_abnormal_count/total_data)*100:.1f}%)', marker='x', s=10)

        # Heavy Abnormal
        heavy_abnormal_data = df[df['Label'] == 'Heavy Abnormal']
        plt.scatter(heavy_abnormal_data['TimeStamp'], heavy_abnormal_data[feature],
                    color='red', label=f'Heavy Abnormal (N={heavy_abnormal_count}, {(heavy_abnormal_count/total_data)*100:.1f}%))', marker='x', s=10)

        # Add vertical lines for spray start and end times
        plt.axvline(spray_start_time, color='black', linestyle='--', linewidth=1.2)
        plt.axvline(spray_end_time, color='black', linestyle='--', linewidth=1.2)

        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
        plt.gca().xaxis.set_major_locator(mdates.MinuteLocator(interval=5)) 
        plt.title(f"{feature} of {nozzle_condition} Nozzle", fontsize=14)
        plt.xlabel("Time", fontsize=12)
        plt.ylabel("Value", fontsize=12)
        plt.legend(fontsize=10)
        plt.grid(alpha=0.3)

    plt.tight_layout()
    plt.show()


In [ ]:
# Bad Nozzle

analyze_spray_data(
    df=pd.read_csv("../data/20250108_aluminum.csv"),
    start_time="20250108T06:26:00.012",
    end_time="20250108T07:04:59.007",
    spray_start_time="20250108T06:36:22.014",
    spray_end_time="20250108T06:55:52.007",
    model=model,
    scaler=scaler,
    pros_param=["Chamber_Pressure", "Main_Gas_Flow", "PF2_Pressure", "PF2_Gas_Flow"],
    timesteps=30,
    slight_abnormal_threshold=slight_abnormal_threshold,  
    heavy_abnormal_threshold=heavy_abnormal_threshold,
    nozzle_condition="Bad Conditioned Polymer"
)


In [ ]:
# Good Nozzle

analyze_spray_data(
    df=pd.read_csv("../data/20250108_aluminum.csv"),
    start_time="20250108T07:24:00.013",
    end_time="20250108T08:06:59.014",
    spray_start_time="20250108T07:36:27.026",
    spray_end_time="20250108T07:55:57.005",
    model=model, 
    scaler=scaler,
    pros_param=["Chamber_Pressure", "Main_Gas_Flow", "PF2_Pressure", "PF2_Gas_Flow"],
    timesteps=30,
    slight_abnormal_threshold=slight_abnormal_threshold,  
    heavy_abnormal_threshold=heavy_abnormal_threshold,
    nozzle_condition="Good Conditioned Polymer"
)
